In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Device set to:", device)

Device set to: cuda


In [3]:
import pandas as pd

df=pd.read_csv('/kaggle/input/20th-aug-images/updated_dataset.csv')

In [4]:
df.head()

,Unnamed: 0.1,Unnamed: 0,Sl No.,text,Humor,Mechanism,Article link,image_url,labels,texts,...,partial_matching_images,pages_with_matching_images,crop_hints,Headline_Response,Combined_Entities_Response,First Part,Split_Headline_Responses,label,6_way_label,path
0,0,2856,NaN,craigslist ad get paid to attend harvard for me,NaN,NaN,NaN,https://external-preview.redd.it/jt6tIxka4W2mj...,NaN,NaN,...,NaN,NaN,NaN,?\n\nSatire often uses irony and exaggeration ...,.\n\n**Related Keywords:**\n\n- Affiliate mark...,craigslist ad get paid,to promote your affiliate products,0,0.0,/content/drive/MyDrive/20th_aug_capstone/image...
1,1,2093,NaN,this cat chillin on an arm rest,NaN,NaN,NaN,https://preview.redd.it/ygvtnvtoqb831.jpg?widt...,NaN,NaN,...,NaN,NaN,NaN,", looking nonchalant and aloof.\n\n**""He's the...",.\n\nThese are all cat breeds.\n\nThese are al...,this cat chillin,' with its human,0,0.0,/content/drive/MyDrive/20th_aug_capstone/image...
2,2,4974,4974.0,Disney Edits Controversial Mandalorian Scene S...,1.0,GP,https://babylonbee.com/news/disney-edits-contr...,https://media.babylonbee.com/articles/article-...,"Nose, Skin, Head, Organ, Human, Toy, Mammal, G...",NaN,...,https://pbs.twimg.com/media/Em-vOW3XcAQYlGT?fo...,https://www.yucelgumruk.com.tr/?w=why-does-the...,"[x: 139\n, x: 730\n, x: 730\ny: 787\n, x: 139\...",This caption is mocking Disney's recent adapta...,", Baby\n\n**Related Keywords:**\n\n- Baby Yoda...",Disney Edits Controversial Mandalorian Scene So,Far\n\nDisney has faced controversy for,1,NaN,/content/drive/MyDrive/20th_aug_capstone/image...
3,3,10121,10123.0,Hero: This Man Refused To Turn To His Neighbor...,1.0,RL,https://babylonbee.com/news/hero-man-refused-t...,https://media.babylonbee.com/articles/article-...,"Forehead, Nose, Cheek, Lip, Chin, Hairstyle, E...","A, A",...,https://get.pxhere.com/photo/man-person-hair-m...,https://www.elastflexespumas.com.br/how-to-tak...,"[x: 174\n, x: 469\n, x: 469\ny: 393\n, x: 174\...",.\n\nSatire often uses irony and exaggeration ...,", Artist, Light, Composition, Background, Fram...",Hero: This Man Refused To Turn,"Away From His Dreams, Captured In",1,NaN,/content/drive/MyDrive/20th_aug_capstone/image...
4,4,10483,NaN,bentley says national leaders should look to a...,NaN,NaN,NaN,https://external-preview.redd.it/cpqJ9WmQhmFRA...,NaN,NaN,...,NaN,NaN,NaN,.\n\nThis caption is mocking the idea that Ala...,**Related Keywords:**\n\n- Public speaking\n- ...,bentley says national leaders should look,to Paul Chan Mo-po for,0,0.0,/content/drive/MyDrive/20th_aug_capstone/image...


In [5]:
df = df.sample(frac=1, random_state=2)


In [6]:
def split_text_by_words(text):
    words = text.split()
    mid = len(words) // 2
    
    first_part = ' '.join(words[:mid])
    second_part = ' '.join(words[mid:])
    
    return first_part, second_part

df[['Part1', 'second_part']] = df['text'].apply(lambda x: pd.Series(split_text_by_words(x)))

In [7]:
df['Split_Headline_Responses']=str(df['Split_Headline_Responses'])

In [8]:
df['path'] = df['path'].str.replace('/content/drive/MyDrive/20th_aug_capstone','/kaggle/input/20th-aug-images')

In [9]:
df2=df

In [10]:
!pip install salesforce-lavis

  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.4/235.4 kB 2.6 MB/s eta 0:00:00
  Installing build dependencies ... - \ | / - \ | / done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... - \ | / done
  Preparing metadata (pyproject.toml) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 MB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.0/377.0 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 3.2 MB/s eta 0:00:

In [11]:
from lavis.models import load_model_and_preprocess
model, vis_processors, txt_processors = load_model_and_preprocess(name="blip2_feature_extractor", model_type="coco", is_eval=True, device=device)

2024-08-20 15:26:11.245624: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-20 15:26:11.245717: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-20 15:26:11.365258: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

100%|██████████| 1.89G/1.89G [00:11<00:00, 184MB/s]


Position interpolate from 16x16 to 26x26


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

100%|██████████| 4.37G/4.37G [00:39<00:00, 119MB/s] 


In [12]:
from PIL import Image
import pandas as pd

multimodal_embeddings = []
image_embeddings = []
text_embeddings = []

for index, row in df.iterrows():
    image_path = row['path']
    image = Image.open(image_path).convert("RGB")
    
    text = row['text']
    text_input = txt_processors["eval"](text)
    
    image_processed = vis_processors["eval"](image).unsqueeze(0).to(device)
    sample = {"image": image_processed, "text_input": [text_input]}
    
    multimodal_emb = model.extract_features(sample).multimodal_embeds[0,0,:] 
    image_emb = model.extract_features(sample, mode="image").image_embeds[0,0,:] 
    text_emb = model.extract_features(sample, mode="text").text_embeds[0,0,:]
    
    multimodal_embeddings.append(multimodal_emb.cpu().numpy())
    image_embeddings.append(image_emb.cpu().numpy())
    text_embeddings.append(text_emb.cpu().numpy())

df['Multimodal Embeddings'] = multimodal_embeddings
df['Image Embeddings'] = image_embeddings
df['Text Embeddings'] = text_embeddings
import numpy as np
multimodal_embeddings =np.array(multimodal_embeddings)
text_embeddings = np.array(text_embeddings)
image_embeddings = np.array(image_embeddings)

In [13]:
import gc
gc.collect()

247

In [14]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 691.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 81.5 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.26.1
    Uninstalling transformers-4.26.1:
      Successfully uninstalled transformers-4.26.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
salesforce-lavis 1.0.2 requires transformers<4.27,>=4.25.0, but you have transformers 4.44.1 which is incompatible.


In [15]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
from scipy.spatial.distance import cosine
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from transformers import BertTokenizer, BertModel
from sentence_transformers import SentenceTransformer
import torch
from tqdm import tqdm

# Load the USE model
use_embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
bert_tokenizer = BertTokenizer.from_pretrained(model_name)
bert_model = BertModel.from_pretrained(model_name)

# Load Sentence-BERT model
sbert_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Function to get USE embedding
def get_use_embedding(texts):
    return use_embed(texts).numpy()

# Function to get BERT embedding
def get_bert_embedding(texts):
    inputs = bert_tokenizer(texts, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

# Function to get Sentence-BERT embedding
def get_sbert_embedding(texts):
    return sbert_model.encode(texts)

def calculate_similarity(emb1, emb2):
    return 1 - cosine(emb1, emb2)

# Function to process data in batches
def process_data_in_batches(df, batch_size=32):
    features = []
    for i in tqdm(range(0, len(df), batch_size)):
        batch = df.iloc[i:i+batch_size]
        
        use_emb1 = get_use_embedding(batch['second_part'].tolist())
        use_emb2 = get_use_embedding(batch['Split_Headline_Responses'].tolist())
        bert_emb1 = get_bert_embedding(batch['second_part'].tolist())
        bert_emb2 = get_bert_embedding(batch['Split_Headline_Responses'].tolist())
        sbert_emb1 = get_sbert_embedding(batch['second_part'].tolist())
        sbert_emb2 = get_sbert_embedding(batch['Split_Headline_Responses'].tolist())
        
        use_similarities = np.array([calculate_similarity(e1, e2) for e1, e2 in zip(use_emb1, use_emb2)])
        bert_similarities = np.array([calculate_similarity(e1, e2) for e1, e2 in zip(bert_emb1, bert_emb2)])
        sbert_similarities = np.array([calculate_similarity(e1, e2) for e1, e2 in zip(sbert_emb1, sbert_emb2)])
        
        batch_features = np.concatenate([use_emb1, use_emb2, bert_emb1, bert_emb2, sbert_emb1, sbert_emb2,
                                         use_similarities.reshape(-1, 1), 
                                         bert_similarities.reshape(-1, 1),
                                         sbert_similarities.reshape(-1, 1)], axis=1)
        features.append(batch_features)
    
    return np.concatenate(features, axis=0)

# Process data in batches
print("Processing data in batches...")
X = process_data_in_batches(df2)
y = df['label'].values

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/transformers/utils/import_utils.py", line 1110, in _get_module
    def is_jumanpp_available():
  File "/opt/conda/lib/python3.10/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1050, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1027, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1006, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 688, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 883, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/opt/conda/lib/python3.10/site-packages/transformers/models/bert/modeling_bert.py", line 48, in <module>
    from ...utils import (
ImportError: cannot import name 'get_torch_version' from 'transformers.utils' (/opt/conda/lib/pytho

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, classification_report
import numpy as np

In [ ]:
# Convert similarity metrics features and labels to PyTorch tensors
X_similarity = torch.tensor(X, dtype=torch.float32)
y_similarity = torch.tensor(y, dtype=torch.float32)

In [ ]:
# Convert multimodal inputs to tensors (assuming these are numpy arrays)
multimodal_inputs = torch.tensor(multimodal_embeddings, dtype=torch.float32)
image_inputs = torch.tensor(image_embeddings, dtype=torch.float32)
text_inputs = torch.tensor(text_embeddings, dtype=torch.float32)

In [ ]:

# Combine all inputs for the BLIP embeddings
combined_blip_inputs = torch.cat((multimodal_inputs, image_inputs, text_inputs), dim=1)

In [ ]:
# Combine similarity metrics features and BLIP embeddings into one dataset
combined_inputs = torch.cat((combined_blip_inputs, X_similarity), dim=1)
labels_combined = torch.tensor(labels, dtype=torch.float32)  # Assuming the labels are the same for both parts


In [ ]:
# Train-test-validation split
X_train, X_temp, y_train, y_temp = train_test_split(combined_inputs, labels_combined, test_size=0.3, random_state=2, stratify=labels_combined)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.6667, random_state=2, stratify=y_temp)


In [ ]:
# Create DataLoaders
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)
test_dataset = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [ ]:
# Define the combined neural network for binary classification
class CombinedBinaryNet(nn.Module):
    def __init__(self, blip_input_size, similarity_input_size):
        super(CombinedBinaryNet, self).__init__()
        combined_input_size = blip_input_size + similarity_input_size
        self.fc1 = nn.Linear(combined_input_size, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = torch.sigmoid(self.fc3(x))
        return x.squeeze()

In [ ]:
# Initialize the model
blip_input_size = combined_blip_inputs.shape[1]
similarity_input_size = X_similarity.shape[1]
model = CombinedBinaryNet(blip_input_size, similarity_input_size)

In [ ]:
# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Early stopping parameters
patience = 10
best_val_loss = float('inf')
counter = 0

In [ ]:
# Train the model
num_epochs = 100
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            val_loss += loss.item()
    
    train_loss /= len(train_loader)
    val_loss /= len(val_loader)
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')
    
    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0
        torch.save(model.state_dict(), 'best_combined_model.pth')
    else:
        counter += 1
        if counter >= patience:
            print(f'Early stopping triggered after epoch {epoch+1}')
            break


In [ ]:
# Load best model
model.load_state_dict(torch.load('best_combined_model.pth'))

In [ ]:
# Evaluate the model
model.eval()
all_preds = []
all_targets = []

In [ ]:
with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        predicted = (outputs > 0.5).float()
        all_preds.extend(predicted.cpu().numpy())
        all_targets.extend(targets.cpu().numpy())

In [ ]:
# Convert to numpy arrays
all_preds = np.array(all_preds)
all_targets = np.array(all_targets)

In [ ]:
# Compute and print accuracy
accuracy = accuracy_score(all_targets, all_preds)
print(f'Test Accuracy: {accuracy:.4f}')

In [ ]:
# Compute and print F1 score
f1 = f1_score(all_targets, all_preds)
print(f'Test F1 Score: {f1:.4f}')

In [ ]:
# Print classification report
print("Classification Report:")
print(classification_report(all_targets, all_preds))
